# Adversarial Debiasing with DiCE

This notebook demonstrates how to use the adversarial debiasing feature in DiCE to increase model fairness.

In [ ]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

import oviya_dice
from oviya_dice import Data, Model, Dice
from oviya_dice.constants import BackEndTypes

from oviya_dice.utils.helpers import load_adult_income_dataset

## Load the Adult Income Dataset

We'll use the UCI Adult Income dataset, which predicts whether income exceeds $50K/yr based on census data.

In [2]:
# Load data
dataset = load_adult_income_dataset()
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [3]:
# Define protected attribute(s)
protected_attributes = ['gender']

# Create a Data object
d = Data(dataframe=dataset, 
         continuous_features=['age', 'hours_per_week'], 
         outcome_name='income',
         protected_attributes=protected_attributes)

## Create and Train a Model with Adversarial Debiasing

Now we'll create a model using the adversarial debiasing approach to mitigate bias.

In [4]:
# Create model with adversarial debiasing
print(dir(BackEndTypes))
model = Model(backend=BackEndTypes.AdversarialDebiasing, 
              model_type="classifier",
              func="ohe-min-max",
              protected_attributes=protected_attributes,
              debias_weight=0.7)

['ALL', 'AdversarialDebiasing', 'Pytorch', 'Sklearn', 'Tensorflow1', 'Tensorflow2', 'VALA', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


/var/folders/y7/g00g5c0s1l9fvqkz8t8k2z740000gn/T/ipykernel_14023/2830132001.py:3: UserWarning: adversarial_debiasing backend not in supported backends sklearn,TF1,TF2,PYT
  model = Model(backend=BackEndTypes.AdversarialDebiasing,


ValueError: should provide either a trained model or the path to a model

In [ ]:
# Extract features, target, and protected attributes
X = dataset.drop(['income', 'gender'], axis=1)
y = dataset['income']
protected = dataset['gender'].map({'Male': 1, 'Female': 0})

# Train the model
model.train_model(X, y, protected, epochs=20, batch_size=64)

## Evaluate Fairness Metrics

Let's evaluate the fairness of our model using various metrics.

In [ ]:
# Evaluate fairness
fairness_metrics = model.evaluate_fairness(X, y, protected)
print("Fairness Metrics:")
for metric, value in fairness_metrics.items():
    print(f"{metric}: {value:.4f}")

## Generate Counterfactual Explanations

Now we'll use DiCE to generate counterfactual explanations for our debiased model.

In [ ]:
# Use DiCE to generate counterfactual explanations
exp = Dice(d, model, method="random")
query_instance = X.iloc[0:1]
counterfactuals = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")

# Display counterfactuals
counterfactuals.visualize_as_dataframe()

## Compare with a Standard Model

Let's compare our debiased model with a standard model without debiasing.

In [ ]:
# Create a standard model without debiasing
standard_model = Model(backend="sklearn", model_type="classifier", func="ohe-min-max")

# Train the standard model
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Prepare the model
numerical = ['age', 'hours_per_week']
categorical = X.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])

# Train the model
standard_model.model = clf.fit(X, y)

In [ ]:
# Create a DiCE explainer for the standard model
standard_exp = Dice(d, standard_model, method="random")

# Generate counterfactuals
standard_counterfactuals = standard_exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")

# Display counterfactuals
standard_counterfactuals.visualize_as_dataframe()

## Evaluate Fairness of Standard Model

Let's evaluate the fairness of our standard model for comparison.

In [ ]:
# Import fairness metrics
from dice_ml.utils.fairness_metrics import (
    demographic_parity_difference,
    equal_opportunity_difference,
    disparate_impact_ratio,
    equalized_odds_difference
)

# Get predictions from standard model
standard_preds = standard_model.model.predict_proba(X)[:, 1]

# Calculate fairness metrics
standard_metrics = {
    'demographic_parity_difference': demographic_parity_difference(standard_preds, protected),
    'equal_opportunity_difference': equal_opportunity_difference(standard_preds, y, protected),
    'disparate_impact_ratio': disparate_impact_ratio(standard_preds, protected),
    'equalized_odds_difference': equalized_odds_difference(standard_preds, y, protected)
}

print("Standard Model Fairness Metrics:")
for metric, value in standard_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nAdversarial Debiased Model Fairness Metrics:")
for metric, value in fairness_metrics.items():
    print(f"{metric}: {value:.4f}")

## Conclusion

In this notebook, we demonstrated how to use adversarial debiasing in DiCE to create fairer models. We compared the fairness metrics between a standard model and an adversarially debiased model, showing how the debiasing approach can help reduce bias in machine learning models.